In [25]:
from src.model import LIFNetwork as lif
import numpy as np
from importlib import reload

reload(lif)

<module 'src.model.LIFNetwork' from '/Users/anthonylee/Documents/GitHub/Vibrotactile-Stimulation/src/model/LIFNetwork.py'>

In [26]:
LIF = lif.LIF_Network(n_neurons=5)
LIF.simulate(epoch_current_input=None)

spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input
spike_t_diff
s_flag
network_W
network_conn
Multiplication
network input


(array([[-52.14192688,  20.        , -50.52717963, -47.46420252,
         -46.95123667],
        [-52.13274218,  20.        , -50.51746359, -47.45781707,
         -46.94524018],
        [-52.12356345,  20.        , -50.50775509, -47.45143593,
         -46.93924771],
        [-52.11439067,  20.        , -50.49805411, -47.44505909,
         -46.93325925],
        [-52.10522386,  20.        , -50.48836066, -47.43868656,
         -46.9272748 ],
        [-52.096063  ,  20.        , -50.47867473, -47.43231832,
         -46.92129436],
        [-52.08690808,  20.        , -50.46899631, -47.42595439,
         -46.91531793],
        [-52.07775912,  20.        , -50.4593254 , -47.41959474,
         -46.9093455 ],
        [-52.06861609,  20.        , -50.44966198, -47.41323939,
         -46.90337707],
        [-52.059479  ,  20.        , -50.44000607, -47.40688832,
         -46.89741264]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0

In [39]:
vec_a = np.random.randint(low=0, high=5, size=3)
mat_a = np.random.randint(low=0, high=5, size=(3, 3))
mat_b = np.random.randint(low=0, high=5, size=(3, 3))

print(vec_a)
print()
print(mat_a)
print()
print(mat_b)
print()

print(np.matmul(vec_a, mat_a))
print(np.matmul(mat_a, vec_a))

## Okay, so the order of matmul does matter, and it is okay to switch them.

[0 1 1]

[[1 0 4]
 [4 4 1]
 [0 3 1]]

[[3 4 1]
 [0 3 4]
 [2 3 2]]

[4 7 2]
[4 5 4]


In [41]:
print(2 * mat_b)

[[6 8 2]
 [0 6 8]
 [4 6 4]]
